<a href="https://colab.research.google.com/github/componavt/topkar-space/blob/main/src/ner/Bert_abbriviations.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Упрщённая версия только для географических аббривиатур

In [ ]:
import pandas as pd
import torch
from transformers import BertTokenizer, BertForSequenceClassification
from sklearn.preprocessing import LabelEncoder
import pickle

class GeographyAbbreviationModel:
    """Упрощенная модель для географических аббревиатур"""

    def __init__(self):
        self.tokenizer = None
        self.model = None
        self.label_encoder = None

    def create_training_data(self):
        """Создает пример данных для географических объектов"""

        data = [
            # Города с разными префиксами
            {"text": "Я живу в г. Москва.", "abbreviation": "г.", "expansion": "город"},
            {"text": "Поеду в г. Санкт-Петербург.", "abbreviation": "г.", "expansion": "город"},
            {"text": "р. Волга красива.", "abbreviation": "р.", "expansion": "река"},
            {"text": "оз. Байкал глубокое.", "abbreviation": "оз.", "expansion": "озеро"},

            # Адреса
            {"text": "ул. Ленина центральная.", "abbreviation": "ул.", "expansion": "улица"},
            {"text": "пр. Мира широкий.", "abbreviation": "пр.", "expansion": "проспект"},
            {"text": "пл. Победы большая.", "abbreviation": "пл.", "expansion": "площадь"},

            # Страны и организации
            {"text": "США - большая страна.", "abbreviation": "США", "expansion": "Соединенные Штаты Америки"},
            {"text": "РФ нашла решение.", "abbreviation": "РФ", "expansion": "Российская Федерация"},
            {"text": "ЕС принял закон.", "abbreviation": "ЕС", "expansion": "Европейский Союз"},
        ]

        df = pd.DataFrame(data)
        df.to_csv('geo_abbreviations.csv', index=False, encoding='utf-8')
        print(f"Создан CSV с {len(df)} географическими примерами")
        return df

    def prepare_dataset(self, csv_path='data_learn.csv'):
        """Подготавливает данные для обучения"""

        try:
            df = pd.read_csv(csv_path, encoding='utf-8')
        except:
            print("CSV не найден, создаю пример...")
            df = self.create_training_data()

        # Простая подготовка текста
        df['input_text'] = df.apply(
            lambda row: f"Текст: {row['text']} | Объект: {row['abbreviation']}",
            axis=1
        )

        # Кодируем метки
        self.label_encoder = LabelEncoder()
        df['label'] = self.label_encoder.fit_transform(df['expansion'])

        print(f"Классы: {list(self.label_encoder.classes_)}")
        return df

    def train(self, df):
        """Обучает модель"""

        # Токенайзер
        self.tokenizer = BertTokenizer.from_pretrained('cointegrated/rubert-tiny2')

        # Модель
        self.model = BertForSequenceClassification.from_pretrained(
            'cointegrated/rubert-tiny2',
            num_labels=len(self.label_encoder.classes_)
        )

        # Простое обучение
        from torch.utils.data import DataLoader, TensorDataset
        import torch.optim as optim

        # Подготовка данных
        inputs = self.tokenizer(
            df['input_text'].tolist(),
            padding=True,
            truncation=True,
            max_length=128,
            return_tensors='pt'
        )

        labels = torch.tensor(df['label'].tolist())

        dataset = TensorDataset(inputs['input_ids'], inputs['attention_mask'], labels)
        dataloader = DataLoader(dataset, batch_size=2, shuffle=True)

        # Обучение
        optimizer = optim.AdamW(self.model.parameters(), lr=2e-5)

        self.model.train()
        for epoch in range(3):  # 3 эпохи
            total_loss = 0
            for batch in dataloader:
                input_ids, attention_mask, batch_labels = batch

                optimizer.zero_grad()
                outputs = self.model(
                    input_ids=input_ids,
                    attention_mask=attention_mask,
                    labels=batch_labels
                )

                loss = outputs.loss
                loss.backward()
                optimizer.step()

                total_loss += loss.item()

            print(f"Эпоха {epoch+1}, Loss: {total_loss/len(dataloader):.4f}")

        # Сохраняем
        self.model.save_pretrained('geo_model')
        self.tokenizer.save_pretrained('geo_model')
        with open('geo_label_encoder.pkl', 'wb') as f:
            pickle.dump(self.label_encoder, f)

        print("Модель сохранена")

    def predict(self, text, abbreviation):
        """Предсказывает географический объект"""

        if self.model is None:
            self.model = BertForSequenceClassification.from_pretrained('geo_model')
            self.tokenizer = BertTokenizer.from_pretrained('geo_model')
            with open('geo_label_encoder.pkl', 'rb') as f:
                self.label_encoder = pickle.load(f)

        input_text = f"Текст: {text} | Объект: {abbreviation}"

        inputs = self.tokenizer(
            input_text,
            return_tensors='pt',
            padding=True,
            truncation=True,
            max_length=128
        )

        with torch.no_grad():
            outputs = self.model(**inputs)
            prediction = torch.argmax(outputs.logits, dim=1).item()

        return self.label_encoder.inverse_transform([prediction])[0]

# Использование
if __name__ == "__main__":
    model = GeographyAbbreviationModel()
    df = model.prepare_dataset()
    model.train(df)

    # Тест
    result = model.predict("Пожня на юго-западе в 300 м.", "м.")
    print(f"Предсказание: {result}")  # улица Ленина

Классы: ['берег', 'болото', 'бор', 'деревня', 'лес', 'метр', 'мост', 'мостки', 'мыс', 'нива', 'озеро', 'река', 'урочище', 'фамилия']


Some weights of BertForSequenceClassification were not initialized from the model checkpoint at cointegrated/rubert-tiny2 and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Эпоха 1, Loss: 2.0770
Эпоха 2, Loss: 1.6511
Эпоха 3, Loss: 1.3784
Модель сохранена
Предсказание: деревня


In [ ]:
!pip install datasets
!pip install evaluate

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 3.6 MB/s eta 0:00:00


Тестовый вариант с более общим подходом

In [ ]:
import torch
import pandas as pd
import numpy as np
import re
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from transformers import (
    BertTokenizer,
    BertForSequenceClassification,
    Trainer,
    TrainingArguments,
    EarlyStoppingCallback
)
from torch.utils.data import Dataset
import evaluate
import warnings
import os
import pickle
from typing import Dict, List, Tuple, Optional, Any
import logging
from dataclasses import dataclass
import math

warnings.filterwarnings('ignore')

# Настройка логирования
logging.basicConfig(
    level=logging.INFO,
    format='%(asctime)s - %(name)s - %(levelname)s - %(message)s'
)
logger = logging.getLogger(__name__)

# ==================== 1. КОНФИГУРАЦИЯ ====================

@dataclass
class Config:
    """Конфигурация обучения"""
    MODEL_NAME: str = 'cointegrated/rubert-tiny2'
    CSV_PATH: str = 'abbreviations_dataset.csv'
    OUTPUT_DIR: str = './abbreviation_model'
    MAX_LENGTH: int = 128  # Уменьшили для более быстрого обучения
    BATCH_SIZE: int = 4    # Уменьшили для небольших данных
    EPOCHS: int = 10       # Уменьшили количество эпох
    TEST_SIZE: float = 0.3  # Увеличили тестовую выборку
    MIN_SAMPLES_PER_CLASS: int = 2  # Минимум примеров на класс
    RANDOM_SEED: int = 42
    SPECIAL_TOKENS: Dict[str, str] = None

    def __post_init__(self):
        if self.SPECIAL_TOKENS is None:
            self.SPECIAL_TOKENS = {
                'start': '[ABBR]',
                'end': '[/ABBR]',
                'sep': '[SEP]'
            }

config = Config()

# ==================== 2. УТИЛИТЫ ДЛЯ РАБОТЫ С АББРЕВИАТУРАМИ ====================

class AbbreviationProcessor:
    """Класс для обработки аббревиатур в тексте"""

    @staticmethod
    def find_all_abbreviations(text: str, min_length: int = 2) -> List[Dict[str, Any]]:
        """Находит все возможные аббревиатуры в тексте."""
        abbreviations = []

        pattern = r'\b(?:[A-ZА-ЯЁ]{2,}|\d+[A-ZА-ЯЁ]+|[A-ZА-ЯЁ]+\d+)[A-ZА-ЯЁ\d]*\b'

        for match in re.finditer(pattern, text):
            abbr = match.group()
            start, end = match.span()

            if len(abbr) < min_length:
                continue

            abbreviations.append({
                'text': abbr,
                'start': start,
                'end': end,
                'context': text[max(0, start-30):min(len(text), end+30)]
            })

        return abbreviations

    @staticmethod
    def mark_specific_abbreviation(
        text: str,
        target_abbr: str,
        target_start: Optional[int] = None
    ) -> Tuple[str, int]:
        """Маркирует конкретную аббревиатуру в тексте специальными токенами."""
        if target_start is not None:
            marked_text = (
                text[:target_start] +
                f"{config.SPECIAL_TOKENS['start']}{target_abbr}{config.SPECIAL_TOKENS['end']}" +
                text[target_start + len(target_abbr):]
            )
            return marked_text, target_start

        pattern = re.compile(r'\b' + re.escape(target_abbr) + r'\b', re.IGNORECASE)
        matches = list(pattern.finditer(text))

        if not matches:
            marked_text = f"{text} {config.SPECIAL_TOKENS['sep']} {target_abbr}"
            return marked_text, len(text) + len(config.SPECIAL_TOKENS['sep']) + 1

        first_match = matches[0]
        start, end = first_match.span()

        marked_text = (
            text[:start] +
            f"{config.SPECIAL_TOKENS['start']}{target_abbr}{config.SPECIAL_TOKENS['end']}" +
            text[end:]
        )

        return marked_text, start

    @staticmethod
    def prepare_training_example(
        text: str,
        abbreviation: str,
        expansion: str,
        position: Optional[int] = None
    ) -> Dict[str, Any]:
        """Подготавливает один пример для обучения."""
        marked_text, abbr_start = AbbreviationProcessor.mark_specific_abbreviation(
            text, abbreviation, position
        )

        return {
            'original_text': text,
            'marked_text': marked_text,
            'abbreviation': abbreviation,
            'expansion': expansion,
            'abbreviation_start': abbr_start,
        }

# ==================== 3. ЧТЕНИЕ И ПОДГОТОВКА ДАННЫХ ====================

def load_and_prepare_data(csv_path: str) -> Tuple[pd.DataFrame, LabelEncoder]:
    """Загружает и подготавливает данные из CSV файла."""

    logger.info(f"Загрузка данных из {csv_path}")

    if not os.path.exists(csv_path):
        logger.warning(f"Файл не найден. Создаю пример данных...")
        return create_balanced_sample_dataset()

    try:
        # Пробуем разные кодировки
        for encoding in ['utf-8', 'cp1251', 'latin1']:
            try:
                df = pd.read_csv(csv_path, encoding=encoding)
                if len(df.columns) >= 3:
                    logger.info(f"Успешно загружен CSV с кодировкой '{encoding}'")
                    break
            except:
                continue

        required_columns = ['text', 'abbreviation', 'expansion']
        missing_columns = [col for col in required_columns if col not in df.columns]

        if missing_columns:
            raise ValueError(f"Отсутствуют обязательные колонки: {missing_columns}")

        initial_count = len(df)
        df = df.dropna(subset=required_columns)

        df['text'] = df['text'].astype(str).str.strip()
        df['abbreviation'] = df['abbreviation'].astype(str).str.strip().str.upper()
        df['expansion'] = df['expansion'].astype(str).str.strip()

        df = df.drop_duplicates(subset=['text', 'abbreviation', 'expansion'])

        logger.info(f"Загружено {len(df)} примеров (удалено {initial_count - len(df)})")

        # Если слишком мало данных, используем пример
        if len(df) < config.MIN_SAMPLES_PER_CLASS * 3:
            logger.warning(f"Слишком мало данных ({len(df)} примеров). Использую пример данных...")
            return create_balanced_sample_dataset()

        processed_examples = []

        for idx, row in df.iterrows():
            position = None
            if 'position' in row and pd.notna(row['position']):
                try:
                    position = int(row['position'])
                except:
                    position = None

            example = AbbreviationProcessor.prepare_training_example(
                text=row['text'],
                abbreviation=row['abbreviation'],
                expansion=row['expansion'],
                position=position
            )

            if 'domain' in row:
                example['domain'] = row['domain']

            processed_examples.append(example)

        processed_df = pd.DataFrame(processed_examples)

        # Фильтруем классы с недостаточным количеством примеров
        class_counts = processed_df['expansion'].value_counts()
        valid_classes = class_counts[class_counts >= config.MIN_SAMPLES_PER_CLASS].index

        if len(valid_classes) < 2:
            logger.warning(f"Достаточно данных только для {len(valid_classes)} классов. Использую пример данных...")
            return create_balanced_sample_dataset()

        processed_df = processed_df[processed_df['expansion'].isin(valid_classes)]

        label_encoder = LabelEncoder()
        processed_df['label_id'] = label_encoder.fit_transform(processed_df['expansion'])

        logger.info(f"Уникальных аббревиатур: {processed_df['abbreviation'].nunique()}")
        logger.info(f"Уникальных расшифровок: {len(label_encoder.classes_)}")

        logger.info("\nРаспределение по классам:")
        for expansion, count in class_counts.items():
            logger.info(f"  {expansion[:40]:40} : {count:3}")

        return processed_df, label_encoder

    except Exception as e:
        logger.error(f"Ошибка при загрузке данных: {e}")
        logger.info("Использую пример данных...")
        return create_balanced_sample_dataset()

def create_balanced_sample_dataset() -> Tuple[pd.DataFrame, LabelEncoder]:
    """Создает сбалансированный пример датасета."""

    logger.info("Создание сбалансированного примера данных...")

    # Создаем небольшой но сбалансированный датасет
    examples = []

    # Базовые аббревиатуры с несколькими примерами каждая
    base_abbreviations = [
        ('API', 'Application Programming Interface', 'IT'),
        ('CPU', 'Central Processing Unit', 'Компьютеры'),
        ('HTTP', 'Hypertext Transfer Protocol', 'IT'),
        ('VPN', 'Virtual Private Network', 'IT'),
    ]

    # Создаем по 2 примера для каждой аббревиатуры
    for abbr, expansion, domain in base_abbreviations:
        examples.append({
            'text': f'Используйте {abbr} для доступа к данным.',
            'abbreviation': abbr,
            'expansion': expansion,
            'domain': domain
        })
        examples.append({
            'text': f'Система работает через {abbr} протокол.',
            'abbreviation': abbr,
            'expansion': expansion,
            'domain': domain
        })

    df = pd.DataFrame(examples)

    processed_examples = []

    for _, row in df.iterrows():
        example = AbbreviationProcessor.prepare_training_example(
            text=row['text'],
            abbreviation=row['abbreviation'],
            expansion=row['expansion']
        )

        if 'domain' in row:
            example['domain'] = row['domain']

        processed_examples.append(example)

    processed_df = pd.DataFrame(processed_examples)

    label_encoder = LabelEncoder()
    processed_df['label_id'] = label_encoder.fit_transform(processed_df['expansion'])

    logger.info(f"Создано {len(processed_df)} примеров для {len(label_encoder.classes_)} классов")

    # Сохраняем пример
    save_df = processed_df[['original_text', 'abbreviation', 'expansion']].copy()
    save_df.to_csv(config.CSV_PATH, index=False, encoding='utf-8')
    logger.info(f"Создан CSV файл: {config.CSV_PATH}")

    return processed_df, label_encoder

# ==================== 4. ДАТАСЕТ ДЛЯ ОБУЧЕНИЯ ====================

class AbbreviationDataset(Dataset):
    """Датасет для обучения модели распознавания аббревиатур"""

    def __init__(
        self,
        texts: List[str],
        labels: List[int],
        tokenizer: BertTokenizer,
        max_length: int = 128
    ):
        self.texts = texts
        self.labels = labels
        self.tokenizer = tokenizer
        self.max_length = max_length

    def __len__(self) -> int:
        return len(self.texts)

    def __getitem__(self, idx: int) -> Dict[str, torch.Tensor]:
        text = self.texts[idx]

        encoding = self.tokenizer.encode_plus(
            text,
            add_special_tokens=True,
            max_length=self.max_length,
            padding='max_length',
            truncation=True,
            return_attention_mask=True,
            return_tensors='pt'
        )

        return {
            'input_ids': encoding['input_ids'].flatten(),
            'attention_mask': encoding['attention_mask'].flatten(),
            'labels': torch.tensor(self.labels[idx], dtype=torch.long)
        }
# ==================== 5. МОДЕЛЬ И МЕТРИКИ ====================

def compute_metrics(p):
    """Вычисление метрик качества"""
    predictions, labels = p
    predictions = np.argmax(predictions, axis=1)

    try:
        accuracy_metric = evaluate.load("accuracy")
        f1_metric = evaluate.load("f1")

        accuracy = accuracy_metric.compute(predictions=predictions, references=labels)['accuracy']
        f1 = f1_metric.compute(predictions=predictions, references=labels, average='weighted')['f1']

        return {"accuracy": accuracy, "f1": f1}
    except:
        # Простой расчет accuracy если evaluate не работает
        accuracy = np.mean(predictions == labels)
        return {"accuracy": accuracy, "f1": accuracy}

# ==================== 6. АДАПТИВНОЕ РАЗДЕЛЕНИЕ ДАННЫХ ====================

def adaptive_train_test_split(df: pd.DataFrame, label_col: str = 'label_id',
                            test_size: float = 0.3, min_test_per_class: int = 1):
    """
    Адаптивное разделение данных, которое гарантирует минимум примеров в тестовой выборке.

    Args:
        df: DataFrame с данными
        label_col: колонка с метками
        test_size: доля тестовой выборки
        min_test_per_class: минимум примеров на класс в тестовой выборке

    Returns:
        train_df, val_df
    """

    # Если данных мало, используем простое разделение
    if len(df) < 20:
        train_df = df.sample(frac=1-test_size, random_state=config.RANDOM_SEED)
        val_df = df.drop(train_df.index)
        return train_df, val_df

    # Пытаемся использовать стратифицированное разделение
    try:
        # Рассчитываем количество примеров для теста
        n_test_total = max(int(len(df) * test_size),
                          len(df[label_col].unique()) * min_test_per_class)

        # Гарантируем что тестовая выборка не слишком большая
        n_test_total = min(n_test_total, len(df) - len(df[label_col].unique()))

        if n_test_total >= len(df):
            # Если данных очень мало, оставляем 1-2 примера для теста
            n_test_total = min(2, len(df) - 1)

        # Используем стратификацию если возможно
        train_df, val_df = train_test_split(
            df,
            test_size=n_test_total/len(df),
            random_state=config.RANDOM_SEED,
            stratify=df[label_col],
            shuffle=True
        )

        return train_df, val_df

    except Exception as e:
        logger.warning(f"Ошибка при стратифицированном разделении: {e}")
        logger.info("Использую простое случайное разделение")

        # Простое разделение
        train_df = df.sample(frac=1-test_size, random_state=config.RANDOM_SEED)
        val_df = df.drop(train_df.index)

        return train_df, val_df

# ==================== 7. ОСНОВНОЙ ПРОЦЕСС ОБУЧЕНИЯ ====================

def train_model() -> Tuple[Any, BertTokenizer, LabelEncoder]:
    """Основная функция обучения модели"""

    logger.info("=" * 60)
    logger.info("НАЧАЛО ОБУЧЕНИЯ МОДЕЛИ")
    logger.info("=" * 60)

    # Загрузка данных
    df, label_encoder = load_and_prepare_data(config.CSV_PATH)

    # Сохраняем label encoder
    with open('label_encoder.pkl', 'wb') as f:
        pickle.dump(label_encoder, f)

    logger.info(f"\nВсего примеров: {len(df)}")
    logger.info(f"Количество классов: {len(label_encoder.classes_)}")

    # Для очень маленьких данных - используем все для обучения
    if len(df) < 10:
        logger.warning(f"Очень мало данных ({len(df)} примеров). Использую все данные для обучения.")
        train_df = df
        val_df = df.iloc[:0]  # Пустая валидационная выборка
    else:
        # Используем адаптивное разделение
        train_df, val_df = adaptive_train_test_split(
            df,
            label_col='label_id',
            test_size=config.TEST_SIZE,
            min_test_per_class=1
        )

    logger.info(f"\nОбучающая выборка: {len(train_df)} примеров")
    logger.info(f"Валидационная выборка: {len(val_df)} примеров")

    # Проверяем распределение
    logger.info("\nРаспределение классов:")
    for exp in label_encoder.classes_:
        train_count = (train_df['expansion'] == exp).sum()
        logger.info(f"  {exp[:30]:30} : {train_count:2}")

    # Загрузка модели и токенайзера
    logger.info(f"\nЗагрузка модели: {config.MODEL_NAME}")
    tokenizer = BertTokenizer.from_pretrained(config.MODEL_NAME)

    # Создаем датасеты
    train_dataset = AbbreviationDataset(
        texts=train_df['marked_text'].tolist(),
        labels=train_df['label_id'].tolist(),
        tokenizer=tokenizer,
        max_length=config.MAX_LENGTH
    )

    if len(val_df) > 0:
        val_dataset = AbbreviationDataset(
            texts=val_df['marked_text'].tolist(),
            labels=val_df['label_id'].tolist(),
            tokenizer=tokenizer,
            max_length=config.MAX_LENGTH
        )
    else:
        val_dataset = None

    # Загружаем модель ПОСЛЕ создания датасетов
    # Это важно для правильной инициализации эмбеддингов
    model = BertForSequenceClassification.from_pretrained(
        config.MODEL_NAME,
        num_labels=len(label_encoder.classes_),
        ignore_mismatched_sizes=True
    )

    # НЕ добавляем специальные токены к эмбеддингам модели
    # Это вызывает ошибку "index out of range"

    # Адаптивные параметры обучения
    n_train = len(train_df)
    n_val = len(val_df) if val_df is not None else 0

    effective_epochs = min(config.EPOCHS, max(3, 50 // max(1, n_train // 2)))
    batch_size = min(config.BATCH_SIZE, max(2, n_train))

    logger.info(f"\nАдаптивные параметры обучения:")
    logger.info(f"  Количество эпох: {effective_epochs}")
    logger.info(f"  Размер батча: {batch_size}")
    logger.info(f"  Всего шагов: {max(1, n_train // batch_size * effective_epochs)}")

    # Определяем стратегию оценки
    eval_strategy = "epoch" if n_val > 0 else "no"

    training_args = TrainingArguments(
        output_dir=config.OUTPUT_DIR,
        overwrite_output_dir=True,
        num_train_epochs=effective_epochs,
        per_device_train_batch_size=batch_size,
        per_device_eval_batch_size=batch_size if n_val > 0 else batch_size,
        warmup_steps=max(1, min(10, n_train // 5)),
        weight_decay=0.0,  # Убираем регуляризацию для маленьких данных
        logging_dir='./logs',
        logging_steps=max(1, min(5, n_train // batch_size)),
        eval_strategy=eval_strategy,
        save_strategy="epoch",
        load_best_model_at_end=n_val > 0,
        metric_for_best_model="accuracy" if n_val > 0 else None,
        greater_is_better=True,
        report_to="none",
        save_total_limit=1,
        seed=config.RANDOM_SEED,
        dataloader_num_workers=0,
        fp16=False,
        gradient_accumulation_steps=1,  # Убираем accumulation steps
        learning_rate=2e-5,  # Добавляем явное указание learning rate
    )

    # Создаем тренер
    trainer = Trainer(
        model=model,
        args=training_args,
        train_dataset=train_dataset,
        eval_dataset=val_dataset if n_val > 0 else None,
        compute_metrics=compute_metrics if n_val > 0 else None,
    )

    logger.info("\n" + "=" * 60)
    logger.info("ЗАПУСК ОБУЧЕНИЯ")
    logger.info("=" * 60)

    try:
        train_result = trainer.train()

        # Сохраняем модель
        trainer.save_model(config.OUTPUT_DIR)
        tokenizer.save_pretrained(config.OUTPUT_DIR)

        logger.info(f"\nМодель сохранена в: {config.OUTPUT_DIR}")

        # Оценка если есть валидационные данные
        if n_val > 0:
            logger.info("\n" + "=" * 60)
            logger.info("ФИНАЛЬНАЯ ОЦЕНКА")
            logger.info("=" * 60)

            try:
                eval_results = trainer.evaluate()
                for key, value in eval_results.items():
                    if isinstance(value, float):
                        logger.info(f"{key:20}: {value:.4f}")
            except Exception as e:
                logger.warning(f"Не удалось вычислить метрики: {e}")

        return trainer, tokenizer, label_encoder

    except Exception as e:
        logger.error(f"Ошибка при обучении: {e}")
        logger.info("Пробую обучить с минимальными параметрами...")

        # Попробуем минимальный вариант
        return train_minimal_model(df, label_encoder)

def train_simple_model(df: pd.DataFrame, label_encoder: LabelEncoder):
    """Простая версия обучения для маленьких данных"""

    logger.info("\n" + "=" * 60)
    logger.info("ПРОСТОЕ ОБУЧЕНИЕ (мало данных)")
    logger.info("=" * 60)

    # Используем все данные для обучения
    tokenizer = BertTokenizer.from_pretrained(config.MODEL_NAME)

    model = BertForSequenceClassification.from_pretrained(
        config.MODEL_NAME,
        num_labels=len(label_encoder.classes_),
        ignore_mismatched_sizes=True
    )

    model.resize_token_embeddings(len(tokenizer))

    # Создаем датасет из всех данных
    dataset = AbbreviationDataset(
        texts=df['marked_text'].tolist(),
        labels=df['label_id'].tolist(),
        tokenizer=tokenizer,
        max_length=config.MAX_LENGTH
    )

    # Очень простые параметры
    training_args = TrainingArguments(
        output_dir=config.OUTPUT_DIR,
        overwrite_output_dir=True,
        num_train_epochs=3,
        per_device_train_batch_size=2,
        per_device_eval_batch_size=2,
        warmup_steps=1,
        weight_decay=0.0,
        logging_dir='./logs',
        logging_steps=1,
        eval_strategy="no",
        save_strategy="epoch",
        report_to="none",
        seed=config.RANDOM_SEED,
        dataloader_num_workers=0,
        fp16=False,
    )

    trainer = Trainer(
        model=model,
        args=training_args,
        train_dataset=dataset,
    )

    try:
        trainer.train()
        trainer.save_model(config.OUTPUT_DIR)
        tokenizer.save_pretrained(config.OUTPUT_DIR)

        logger.info(f"\nМодель сохранена в: {config.OUTPUT_DIR}")

        return trainer, tokenizer, label_encoder
    except Exception as e:
        logger.error(f"Ошибка при простом обучении: {e}")
        raise

# ==================== 8. КЛАСС ДЛЯ ПРЕДСКАЗАНИЙ ====================

class AbbreviationPredictor:
    """Класс для предсказания расшифровок аббревиатур"""

    def __init__(self, model_dir: str = None):
        if model_dir is None:
            model_dir = config.OUTPUT_DIR

        if not os.path.exists(model_dir):
            raise FileNotFoundError(f"Директория модели не найдена: {model_dir}")

        self.tokenizer = BertTokenizer.from_pretrained(model_dir)

        special_tokens_dict = {
            'additional_special_tokens': list(config.SPECIAL_TOKENS.values())
        }
        self.tokenizer.add_special_tokens(special_tokens_dict)

        self.model = BertForSequenceClassification.from_pretrained(model_dir)
        self.model.eval()
        self.model.resize_token_embeddings(len(self.tokenizer))

        # Загрузка label encoder
        encoder_path = os.path.join(model_dir, 'label_encoder.pkl')
        if os.path.exists(encoder_path):
            with open(encoder_path, 'rb') as f:
                self.label_encoder = pickle.load(f)
        elif os.path.exists('label_encoder.pkl'):
            with open('label_encoder.pkl', 'rb') as f:
                self.label_encoder = pickle.load(f)
        else:
            # Создаем простой encoder если не найден
            logger.warning("Label encoder не найден. Создаю новый...")
            self.label_encoder = LabelEncoder()
            # Нужно будет дообучить или использовать предопределенные классы

        self.classes = getattr(self.label_encoder, 'classes_', [])
        logger.info(f"Модель загружена. Количество классов: {len(self.classes)}")

    def predict_single(
        self,
        text: str,
        abbreviation: str,
        position: Optional[int] = None,
        top_k: int = 3
    ) -> Dict[str, Any]:
        """Предсказание расшифровки для одной аббревиатуры в тексте."""

        # Маркируем аббревиатуру
        marked_text, abbr_start = AbbreviationProcessor.mark_specific_abbreviation(
            text, abbreviation, position
        )

        # Токенизация
        inputs = self.tokenizer(
            marked_text,
            return_tensors="pt",
            max_length=config.MAX_LENGTH,
            padding="max_length",
            truncation=True
        )

        # Предсказание
        with torch.no_grad():
            outputs = self.model(**inputs)
            probabilities = torch.nn.functional.softmax(outputs.logits, dim=-1)[0]

        # Получаем топ-K предсказаний
        top_k = min(top_k, len(self.classes))
        top_probs, top_indices = torch.topk(probabilities, top_k)

        predictions = []
        for prob, idx in zip(top_probs, top_indices):
            if len(self.classes) > 0:
                expansion = self.label_encoder.inverse_transform([idx.cpu().numpy()])[0]
            else:
                expansion = f"Class_{idx.item()}"

            predictions.append({
                'expansion': expansion,
                'confidence': prob.item(),
                'label_id': idx.item()
            })

        return {
            'text': text,
            'abbreviation': abbreviation,
            'prediction': predictions[0]['expansion'],
            'confidence': predictions[0]['confidence'],
            'all_predictions': predictions,
        }

# ==================== 9. ТЕСТИРОВАНИЕ ====================

def run_simple_tests(predictor: AbbreviationPredictor):
    """Простое тестирование модели"""

    logger.info("\n" + "=" * 60)
    logger.info("ТЕСТИРОВАНИЕ МОДЕЛИ")
    logger.info("=" * 60)

    test_cases = [
        ("Используйте API для доступа к данным.", "API"),
        ("Процессор CPU работает быстро.", "CPU"),
        ("Сайт использует HTTP протокол.", "HTTP"),
        ("Подключитесь через VPN.", "VPN"),
    ]

    for text, abbr in test_cases:
        logger.info(f"\nТекст: {text}")
        logger.info(f"Аббревиатура: {abbr}")

        try:
            result = predictor.predict_single(text, abbr)
            logger.info(f"Предсказание: {result['prediction']}")
            logger.info(f"Уверенность: {result['confidence']:.2%}")
        except Exception as e:
            logger.error(f"Ошибка: {e}")

# ==================== 10. ОСНОВНОЙ БЛОК ====================

def main():
    """Основная функция"""

    print("\n" + "=" * 60)
    print("СИСТЕМА РАСШИФРОВКИ АББРЕВИАТУР")
    print("=" * 60)

    # Проверяем наличие GPU
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    logger.info(f"Используется устройство: {device}")

    # Создаем CSV если его нет
    if not os.path.exists(config.CSV_PATH):
        logger.info(f"Создаю пример CSV файла: {config.CSV_PATH}")
        create_balanced_sample_dataset()

    try:
        # Обучаем модель
        trainer, tokenizer, label_encoder = train_model()

        # Создаем предсказатель
        predictor = AbbreviationPredictor(config.OUTPUT_DIR)

        # Тестируем
        run_simple_tests(predictor)

        print("\n" + "=" * 60)
        print("ОБУЧЕНИЕ ЗАВЕРШЕНО УСПЕШНО!")
        print("=" * 60)

        # Пример использования
        print("\nПример использования:")
        print("```python")
        print("from your_module import AbbreviationPredictor")
        print()
        print("predictor = AbbreviationPredictor()")
        print('result = predictor.predict_single("Используйте API для доступа", "API")')
        print('print(f"Расшифровка: {result[\'prediction\']}")')
        print("```")

        return predictor

    except Exception as e:
        logger.error(f"Критическая ошибка: {e}")
        print("\n" + "=" * 60)
        print("ВОЗНИКЛИ ПРОБЛЕМЫ")
        print("=" * 60)
        print("\nРекомендации:")
        print("1. Убедитесь что файл abbreviations_dataset.csv существует")
        print("2. Проверьте что в файле есть колонки: text, abbreviation, expansion")
        print("3. Добавьте больше данных (минимум 2 примера на класс)")
        print("4. Убедитесь что установлены все зависимости")

        return None

if __name__ == "__main__":
    predictor = main()


СИСТЕМА РАСШИФРОВКИ АББРЕВИАТУР


Some weights of BertForSequenceClassification were not initialized from the model checkpoint at cointegrated/rubert-tiny2 and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Epoch,Training Loss,Validation Loss,Accuracy,F1
1,2.078200,2.096471,0.166667,0.083333
2,2.050600,2.100928,0.000000,0.000000
3,2.012300,2.107200,0.000000,0.000000
4,2.040900,2.112417,0.000000,0.000000
5,2.034800,2.116604,0.000000,0.000000
6,1.965900,2.119481,0.000000,0.000000
7,2.002100,2.121349,0.000000,0.000000
8,2.014400,2.122189,0.000000,0.000000


The new embeddings will be initialized from a multivariate normal distribution that has old embeddings' mean and covariance. As described in this article: https://nlp.stanford.edu/~johnhew/vocab-expansion.html. To disable this, use `mean_resizing=False`



ОБУЧЕНИЕ ЗАВЕРШЕНО УСПЕШНО!

Пример использования:
```python
from your_module import AbbreviationPredictor

predictor = AbbreviationPredictor()
result = predictor.predict_single("Используйте API для доступа", "API")
print(f"Расшифровка: {result['prediction']}")
```
